In [4]:
import pickle
import networkx as nx

In [3]:
with open("migration_graph.gpickle", "rb") as f:
    G = pickle.load(f)

with open("migration_data.pkl", "rb") as f:
    data = pickle.load(f)
key_nodes_df = data["key_nodes_df"]
edges_df = data["edges_df"]
temporal_changes = data["temporal_changes"]

In [5]:
# --- Reciprocity ---

# Overall reciprocity of the directed migration graph
overall_recip = nx.reciprocity(G)
print(f"Overall reciprocity: {overall_recip:.3f}")

# Node-level reciprocity (for key nodes only, if you like)
node_recip = nx.reciprocity(G, nodes=key_nodes_df['cluster_id'].tolist())
print("\nSample node reciprocity (first 10):")
for nid, r in list(node_recip.items())[:10]:
    print(f"  node {nid}: {r:.3f}" if r is not None else f"  node {nid}: None (isolated)")

# --- Assortativity ---

# 1) Degree assortativity (how similar degrees are along edges)
deg_assort = nx.degree_assortativity_coefficient(G, x='out', y='in', weight='Weight')
print(f"\nDegree assortativity (out-degree vs in-degree, weighted): {deg_assort:.3f}")

# 2) Attribute assortativity by state (or region)
# Make sure 'state' (or whatever you used) is a node attribute
# e.g., nx.set_node_attributes(G, dict(zip(key_nodes_df['cluster_id'], key_nodes_df['state'])), 'state')
attr_assort_state = nx.attribute_assortativity_coefficient(G, 'state')
print(f"Attribute assortativity by state: {attr_assort_state:.3f}")

# 3) Optional: numeric assortativity by latitude/longitude
# (requires lat/lon stored as node attributes)
lat_assort = nx.numeric_assortativity_coefficient(G, 'lat')
lon_assort = nx.numeric_assortativity_coefficient(G, 'lon')
print(f"Numeric assortativity by latitude: {lat_assort:.3f}")
print(f"Numeric assortativity by longitude: {lon_assort:.3f}")


Overall reciprocity: 0.004

Sample node reciprocity (first 10):
  node 50: 0.000
  node 140: 0.000
  node 243: 0.000
  node 38: 0.000
  node 119: 0.000
  node 67: 0.000
  node 115: 0.000
  node 51: 0.000
  node 333: 0.000
  node 53: 0.000

Degree assortativity (out-degree vs in-degree, weighted): 0.407
Attribute assortativity by state: nan


/Users/manasi/Library/CloudStorage/GoogleDrive-manasiagarwal29@gmail.com/My Drive/Notability/Semester 1/Network Science/Final Project/venv/lib/python3.12/site-packages/networkx/algorithms/assortativity/correlation.py:282: RuntimeWarning: invalid value encountered in scalar divide
  r = (t - s) / (1 - s)


Numeric assortativity by latitude: 0.878
Numeric assortativity by longitude: 0.120


The overal reciprocity of the network is 0.4 % so our migration netowork is mostly one sided.
 Degree assortativity of about 0.41 indicates moderate positive assortativity, so high‑degree stopover hubs tend to connect to other high‑degree hubs, forming a relatively cohesive “core” of heavily used sites